In [1]:
%matplotlib inline

このユニットでは、機械学習の一般的なタスクのためのAPIを実行します。各セクションのリンクを参照して、より深く掘り下げていきます。

## データを扱う

PyTorch には `torch.utils.data.DataLoader` と `torch.utils.data.Dataset` という2つの[データを扱うためのプリミティブ](https://pytorch.org/docs/stable/data.html)があります。`Dataset` には、サンプルとそれに対応するラベルが格納され、`DataLoader` は `Dataset` にイテレート可能なデータをラップします。

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

PyTorchには、[TorchText](https://pytorch.org/text/stable/index.html), [TorchVision](https://pytorch.org/vision/stable/index.html), [TorchAudio](https://pytorch.org/audio/stable/index.html)などのドメインに特化したライブラリがあり、これらにはデータセットが含まれています。このチュートリアルでは、TorchVisionのデータセットを使います。

`Torchvision.datasets`モジュールには、CIFAR, COCO ([full list here](https://pytorch.org/docs/stable/torchvision/datasets.html))のような、多くの実世界のビジョンデータの `Dataset` オブジェクトが含まれています。このチュートリアルでは、**FashionMNIST**データセットを使用します。TorchVisionの `Dataset` には2つの引数があります。それは `transform` と `target_transform` で、それぞれサンプルとラベルを変更します。

In [3]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

`DataLoader` の引数として `Dataset` を渡します。これはデータセットのイテレート可能性をラッピングしたもので、自動バッチ処理、サンプリング、シャッフル、マルチプロセスデータローディングをサポートしています。ここでは、バッチサイズを64と定義しています。つまり、データローダの反復可能な各要素は、64個の特徴とラベルのバッチを返すことになります。

In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


## モデルの作成
PyTorchでニューラルネットワークを定義するには、[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html)を継承したクラスを作成します。`__init__` 関数でネットワークのレイヤーを定義し、`forward` 関数でデータがネットワークをどのように通過するかを指定します。ニューラルネットワークの演算を高速化するために、可能であればGPUに移します。

In [5]:
# Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# Define model


class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28 * 28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits


model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


## モデルパラメータの最適化

モデルを学習するためには、[損失関数](https://pytorch.org/docs/stable/nn.html#loss-functions>)と[オプティマイザー](https://pytorch.org/docs/stable/optim.html)が必要です。

In [6]:
loss_fn = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

1つの学習ループの中で、モデルは（バッチで与えられた）学習データセットに対して予測を行い、予測誤差をバックプロパゲーションしてモデルのパラメータを調整します。

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

また、テストデータセットに対してモデルの性能を確認し、学習していることを確認します。

In [8]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

学習プロセスは、いくつかの反復（*エポック*）で行われます。各エポックの間に、モデルはより良い予測をするためのパラメータを学習します。エポックごとにモデルの精度と損失を表示します。エポックごとに精度が上がり、損失が減ることを期待しています。

In [9]:
epochs = 15
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.301571  [    0/60000]
loss: 2.294775  [ 6400/60000]
loss: 2.283246  [12800/60000]
loss: 2.277582  [19200/60000]
loss: 2.269792  [25600/60000]
loss: 2.267385  [32000/60000]
loss: 2.252189  [38400/60000]
loss: 2.243959  [44800/60000]
loss: 2.244788  [51200/60000]
loss: 2.225713  [57600/60000]
Test Error: 
 Accuracy: 43.7%, Avg loss: 0.034918 

Epoch 2
-------------------------------
loss: 2.233778  [    0/60000]
loss: 2.217471  [ 6400/60000]
loss: 2.184243  [12800/60000]
loss: 2.183712  [19200/60000]
loss: 2.157760  [25600/60000]
loss: 2.178869  [32000/60000]
loss: 2.141837  [38400/60000]
loss: 2.131330  [44800/60000]
loss: 2.145938  [51200/60000]
loss: 2.095896  [57600/60000]
Test Error: 
 Accuracy: 41.4%, Avg loss: 0.032912 

Epoch 3
-------------------------------
loss: 2.124473  [    0/60000]
loss: 2.085369  [ 6400/60000]
loss: 2.015752  [12800/60000]
loss: 2.021131  [19200/60000]
loss: 1.975584  [25600/60000]
loss: 2.031260  [32000/600

最初は精度があまりよくありません（それでもいいんです！）。もっと多くの `epochs` でループを実行するか、`learning_rate` をより大きな数値に調整してみてください。また、私たちが選んだモデル構成が、この種の問題に最適なものではないかもしれません（そうではありません）。この後の講座では、視覚問題に有効なモデルの形状について、より深く掘り下げていきます。

モデルの保存
-------------
モデルを保存する一般的な方法は、内部状態辞書（モデルのパラメータを含む）をシリアル化することです。

In [ ]:
torch.save(model.state_dict(), "data/model.pth")
print("Saved PyTorch Model State to model.pth")

ローディングモデル
----------------------------

モデルをロードするプロセスでは、モデルの構造を再作成し、そこに状態辞書をロードします。
ステートディクショナリーをロードします。

In [ ]:
model = NeuralNetwork()
model.load_state_dict(torch.load("data/model.pth"))

In [ ]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')